In [ ]:
! pip instal torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import torch
from tensorflow.keras.models import load_model
from google.colab.patches import cv2_imshow

class  FishFreshnessModel:
  def __init__(self, detection_model_path, classification_model_path, classify_fish_path):
    # Muat model YOLOv5 untuk pendeteksian mata mandiri
    self.detection_model = torch.hub.load('ultralytics/yolov5', 'custom', path=detection_model_path)
    # Muat model klasifikasi terlatih mandiri
    self.classification_model = load_model(classification_model_path)
    # Muat model untuk mengklasifikasikan diri ikan
    self.classify_fish_model = load_model(classify_fish_path)


  def preprocess_image(self, image):
    # Preprocess the image to fit the classification model input
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 128))
    img = img/255.0
    img = np.expand_dims(img, axis=0)
    return  img

  def detect_eye_and_classify_freshness(self, image_path):
    # Read the image
    img = cv2.imread(image_path)
    if img is None:
      raise FileNotFoundError(f"Gambar di jalur'{image_path}' tidak ditemukan atau tidak dapat dibaca.")

    # Perform object detection to find the fish eye
    results = self.detection_model(img)

    # Check if any detections were made
    if len(results.xyxy[0]) == 0:
      raise ValueError("Tidak ada mata ikan yang terdeteksi pada gambar.")

    # Loop over the detections
    for det in results.xyxy[0]:
      xmin, ymin, xmax, ymax, conf, cls = det

      # Crop the detected eye region
      cropped_eye = img [int(ymin): int(ymax), int (xmin):int(xmax)]

      # Preprocess the cropped eye image
      processed_eye = self.preprocess_image(cropped_eye)

      # Classify the freshness of the fish
      prediction = self.classification_model.predict(processed_eye)
      predicted_class = np.argmax(prediction, axis=1)

      # Map prediction to class label
      class_labels =  ['Fresh', 'Not Fresh']
      freshness = class_labels[predicted_class[0]]

      # Preprocess image for fish kind classification
      processed_image = self.preprocess_image(img)

      # Classify the kind of fish
      fish_prediction = self.classify_fish_model.predict(processed_image)
      fish_class = np.argmax(fish_prediction, axis=1)

      fish_labels= ['bawal putih' , 'kakap merah' , 'tenggiri' , 'kembung' , 'kerapu' , 'gole gole' , 'gembolo' , 'cakalang' , 'belato' , 'tongkol']
      fish_kind = fish_labels[fish_class[0]]

      # Draw the bounding box and label on the original image
      cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (255, 0, 0), 2)
      cv2.putText(img, freshness, (int(xmin), int(ymin)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
      cv2.putText(img, f'Jenis:{fish_class} ', (int(xmin), int(ymin)-40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36, 255, 12), 2)

    # Display the image with the result using cv2_imshow for Colab
    cv2_imshow(img)

    return freshness, fish_kind

# Tetapkan jalur yang benar untuk model
detection_model_path = '/content/drive/MyDrive/datasetCapstone/best.pt'
classification_model_path = '/content/drive/MyDrive/datasetCapstone/fish_freshness_model_1.h5'
classify_fish_path = '/content/drive/MyDrive/datasetCapstone/saved_model/my_model_klasifikasi_128.h5'

# Buat instance model gabungan
combined_model = FishFreshnessModel(detection_model_path, classification_model_path, classify_fish_path)

# Uji model gabungan
test_image_path =  '/content/drive/MyDrive/datasetCapstone/gembolo2.jpeg'
freshness, fish_kind = combined_model.detect_eye_and_classify_freshness(test_image_path)
print (f'Freshness: {freshness}, Jenis: {fish_kind}')